In [40]:
import os
from subprocess import check_output, Popen
import time
from pathlib import Path
import itertools
import pandas as pd
import tempfile
from numpy import mean, std

In [3]:
"""
CPU name:	Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
CPU type:	Intel Coffeelake processor
CPU stepping:	10
"""
# likwid-bench -t peakflops -W N:16kB:1
peak_MFlops = [5178.29, 5175.56, 5175.70, 5169.06, 5160.98]
# likwid-bench -t peakflops_avx -W N:16kB:1
peak_MFlops_avx = [20621.23, 20631.51, 20569.54, 20616.68, 20632.56]
# likwid-bench -t peakflops_avx_fma -W N:16kB:1
peak_MFlops_avx_fma = [41211.03, 41277.69, 41240.11, 41260.63, 41245.61]

In [5]:
def working_group(mem, units, threads):
    mem = float(mem)
    threads = int(threads)
    assert threads > 0
    return f"-W N:{mem}{units}:{threads}"

In [36]:
def try_parse(s, t):
    try:
        return t(s)
    except ValueError:
        return None
    
def try_parse_int(s):
    return try_parse(s, int)

def try_parse_float(s):
    return try_parse(s, float)

def try_parse_size(l):
    multiplier = 1
    if l[-1] == "MB":
        multiplier = 1024
    size_kB = try_parse_int(l[1])
    if size_kB is None:
        return None
    else:
        return multiplier * size_kB
    
def try_parse_flops(l):
    return try_parse_float(l[1])

In [28]:
def get_topo():
    command = "likwid-topology"
    f = tempfile.TemporaryFile()
    topo_proc = Popen(command, shell=True, stdout=f)
    topo_proc.wait()
    f.seek(0)
    raw_output = f.read()
    lines = raw_output.decode("utf-8").strip().split("\n")
    cache_info_lines = [line.split() for line in lines if "Size" in line]
    cache_info_kB = [try_parse_size(l) for l in cache_info_lines]
    return cache_info_kB

In [60]:
def bench_peakflops(mem, units, threads, trials=5):
    # TODO: FIXME
    # I know I should be using enums, but CBA right now sorry
    benchmark_variants = ["peakflops" for _ in range(3)]
    benchmark_variants[1] += "_avx"
    benchmark_variants[2] += "_avx_fma"
    wg = working_group(mem, units, threads)
    cmd_base = "likwid-bench -t"
    peak_Mflops = []
    for bvar in benchmark_variants:
        cmd = f"{cmd_base} {bvar} {wg}"
        print(cmd)
        f = tempfile.TemporaryFile()
        for _ in range(trials):
            p = Popen(cmd, shell=True, stdout=f)
            p.wait()
        f.seek(0)
        output_lines = f.read().decode("utf-8").strip().split("\n")
        relevant_lines = [l.split() for l in lines if "MFlop" in l]
        print(relevant_lines)
        variant_Mflops = [try_parse_flops(l) for l in relevant_lines]
        print(variant_Mflops)
        peak_Mflops.append(variant_Mflops)
        f.close()
    return peak_MFlops
#     command += " "
#     command += wg
#     f = tempfile.TemporaryFile()
#     bench_proc = Popen(command, shell=True, stdout=f)
#     bench_proc.wait()
#     f.seek(0)
#     raw_output = f.read()
#     lines = raw_output.decode("utf-8").strip().split("\n")
#     return lines

In [58]:
cache_info_lines
buffer_sizes = [m/2 for m in cache_info_lines]

In [61]:
Mflops_data = bench_peakflops(buffer_sizes[0], "kB", 1)

likwid-bench -t peakflops -W N:16.0kB:1


Running without Marker API. Activate Marker API with -m on commandline.
Running without Marker API. Activate Marker API with -m on commandline.
Running without Marker API. Activate Marker API with -m on commandline.
Running without Marker API. Activate Marker API with -m on commandline.
Running without Marker API. Activate Marker API with -m on commandline.


[]
[]
likwid-bench -t peakflops_avx -W N:16.0kB:1


Running without Marker API. Activate Marker API with -m on commandline.
Running without Marker API. Activate Marker API with -m on commandline.
Running without Marker API. Activate Marker API with -m on commandline.


KeyboardInterrupt: 

In [51]:
Mflops_data

[5178.29, 5175.56, 5175.7, 5169.06, 5160.98]

In [ ]:
flops_cmd = "likwid-bench -t peakflops"
bw_cmd = "likwid-bench -t load"

In [4]:
Popen?